음성을 감정 분류 값으로 만드는 코드

해당 모델은 코드를 벡터화 해서 감정분류까지 해주는 코드이다.

별도의 k-wav2vec가 별도의 wav2vec은 추가로 적용해서 할 것

In [ ]:
# 필요한 패키지 설치
!pip install regex==2024.11.6 \
numpy==1.26.4 \
setuptools==75.6.0 \
tokenizers==0.21.0 \
tqdm==4.67.1 \
transformers==4.47.0 \
librosa

In [18]:
import os
import torch
import numpy as np
import soundfile as sf
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)
import torch.nn as nn
import librosa

In [19]:
# Regression Head 정의
# - 마지막 레이어의 임베딩을 입력받아 arousal, dominance, valence를 예측하기 위한 레이어
class RegressionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        # 은닉 벡터 크기: config.hidden_size
        # 출력 레이어는 3차원 (arousal, dominance, valence)로 매핑
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)       # 드롭아웃: 과적합 방지
        x = self.dense(x)         # 선형 변환
        x = torch.tanh(x)         # 활성화 함수: tanh
        x = self.dropout(x)       # 다시 드롭아웃
        x = self.out_proj(x)      # 최종 출력층 -> 감정 값(3차원)
        return x

In [20]:
# EmotionModel 정의
# - Wav2Vec2 모델을 상속받아 Emotion Regression Head를 붙인 모델
class EmotionModel(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        # Pretrained Wav2Vec2 모델 불러오기
        self.wav2vec2 = Wav2Vec2Model(config)
        # 감정 예측 레이어
        self.classifier = RegressionHead(config)
        # 가중치 초기화
        self.init_weights()

    def forward(self, input_values):
        # Wav2Vec2 모델에 입력: (batch, time)
        outputs = self.wav2vec2(input_values)
        # outputs[0]은 (batch, time, hidden_size) 형태의 last hidden states
        hidden_states = outputs[0]

        # 시간축에 대해 평균 풀링 -> (batch, hidden_size)
        hidden_states = torch.mean(hidden_states, dim=1)

        # 평균 풀링한 벡터를 Regression Head로 통과 -> (batch, 3)
        logits = self.classifier(hidden_states)

        # hidden_states: 임베딩, logits: 감정 예측값
        return hidden_states, logits

In [21]:
# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델 및 프로세서 로드
# 사용 모델: audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim
model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = EmotionModel.from_pretrained(model_name).to(device)

In [22]:
# process_func 함수
# 주어진 waveform과 샘플링 레이트를 받아 모델 추론을 수행하는 함수.
# embeddings=True 시: hidden_states 반환
# embeddings=False 시: logits 반환
def process_func(waveform: np.ndarray, sampling_rate: int, embeddings: bool = False) -> np.ndarray:
    """
    오디오 신호에 대해 Wav2Vec2 기반 감정 추론을 수행하는 함수.
    Args:
        waveform (np.ndarray): 오디오 신호 (shape: (1, samples))
        sampling_rate (int): 오디오 샘플링 레이트 (예: 16kHz)
        embeddings (bool): True면 임베딩(hidden_states) 반환, False면 예측 감정(logits) 반환

    Returns:
        np.ndarray: 감정 예측값(배열) 또는 임베딩(배열)
    """

    # 1. Processor를 통해 입력값 전처리
    #    return_tensors="pt"로 파이토치 텐서 반환
    inputs = processor(waveform, sampling_rate=sampling_rate, return_tensors="pt")
    input_values = inputs["input_values"].to(device)

    # 2. 모델 추론 (no_grad로 백프로파게이션 비활성)
    with torch.no_grad():
        hidden_states, logits = model(input_values)

    # 3. embeddings 파라미터에 따라 반환값 선택
    output = hidden_states if embeddings else logits

    # CPU로 이동한 뒤 numpy로 변환
    return output.cpu().numpy()

In [23]:
# resample_audio 함수
def resample_audio(filepath, target_sampling_rate=16000):
    """
    오디오 파일을 지정된 샘플링 레이트로 변환합니다.

    Args:
        filepath (str): 오디오 파일 경로
        target_sampling_rate (int): 변환할 샘플링 레이트 (기본값: 16000)

    Returns:
        np.ndarray: 변환된 오디오 데이터
        int: 변환된 샘플링 레이트
    """
    try:
        # 원본 오디오 로드
        waveform, original_sr = librosa.load(filepath, sr=None)
        print(f"Original sampling rate: {original_sr}")

        # 샘플링 레이트 변환
        if original_sr != target_sampling_rate:
            waveform = librosa.resample(waveform, orig_sr=original_sr, target_sr=target_sampling_rate)
            print(f"Resampled to {target_sampling_rate} Hz")
        else:
            print("Sampling rate already matches the target rate.")

        return waveform, target_sampling_rate
    except Exception as e:
        print(f"Error resampling {filepath}: {e}")
        return None, None

In [24]:
# analyze_all_files_with_resampling 함수
def analyze_all_files_with_resampling(directory: str):
    """
    지정된 디렉토리 내 모든 .wav 파일을 샘플링 레이트 변환 후 분석.
    """
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return

    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        if filename.lower().endswith(".wav"):
            try:
                print(f"Processing file: {filename}")

                # 오디오 샘플링 레이트 변환
                waveform, sr = resample_audio(filepath, target_sampling_rate=16000)
                if waveform is None:
                    continue

                # 모델에 입력 형식으로 변환
                waveform = waveform.reshape(1, -1).astype(np.float32)

                # 감정 예측 수행
                predictions = process_func(waveform, sr, embeddings=False)
                print(f"Predicted Arousal, Dominance, Valence for {filename}: {predictions[0]}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

In [25]:
# 메인 실행 부분
if __name__ == "__main__":
    # 분석할 디렉토리 이름 설정: 여기서는 "voice"
    # voice 폴더 안에 .wav 파일들 넣어두기
    directory = "voice"
    analyze_all_files_with_resampling(directory)

Processing file: temp2.wav
Original sampling rate: 48000
Resampled to 16000 Hz
Predicted Arousal, Dominance, Valence for temp2.wav: [0.36510953 0.43488592 0.51708657]
Processing file: temp.wav
Original sampling rate: 48000
Resampled to 16000 Hz
Predicted Arousal, Dominance, Valence for temp.wav: [0.5687486 0.5290034 0.5430766]


수치들을 3차원 값으로 보여주는 코드

In [27]:
!pip install plotly

In [29]:
import plotly.graph_objects as go

def interactive_3d_plot(emotion_data, filenames):
    """
    예측된 감정 값(Arousal, Dominance, Valence)을 대화형 3D 그래프로 시각화.

    Args:
        emotion_data (list of list): [[Arousal, Dominance, Valence], ...] 형식의 예측 값 리스트
        filenames (list of str): 각 데이터에 해당하는 파일 이름 리스트
    """
    # 예측 값 분리
    arousal = [data[0] for data in emotion_data]
    dominance = [data[1] for data in emotion_data]
    valence = [data[2] for data in emotion_data]

    # 3D 그래프 생성
    fig = go.Figure()

    # 데이터 추가
    fig.add_trace(go.Scatter3d(
        x=arousal,
        y=dominance,
        z=valence,
        mode='markers+text',
        marker=dict(
            size=5,  # 점 크기 줄이기
            color='blue',  # 점 색상
            opacity=0.8
        ),
        text=filenames,  # 파일 이름 레이블
        textposition="top center"
    ))

    # 레이아웃 설정
    fig.update_layout(
        title='3D Emotion Graph',
        scene=dict(
            xaxis=dict(title='Arousal', range=[0, 1]),  # X축 범위 0~1
            yaxis=dict(title='Dominance', range=[0, 1]),  # Y축 범위 0~1
            zaxis=dict(title='Valence', range=[0, 1]),  # Z축 범위 0~1
        ),
        margin=dict(l=0, r=0, b=0, t=40)  # 그래프 여백 설정
    )

    # 그래프 표시
    fig.show()

# 예제 데이터
emotion_predictions = [
    [0.5687486, 0.5290034, 0.5430766],  # temp2.wav
    [0.4568457, 0.6370042, 0.5901238],  # temp.wav
]
file_names = ["temp2.wav", "temp.wav"]

# 대화형 3D 그래프 표시
interactive_3d_plot(emotion_predictions, file_names)

In [30]:
import plotly.graph_objects as go
from sklearn.cluster import KMeans
import numpy as np

def cluster_and_plot_3d(emotion_data, filenames, n_clusters=3):
    """
    감정 데이터에 대해 K-Means 군집화를 수행하고 결과를 3D 그래프로 시각화.

    Args:
        emotion_data (list of list): [[Arousal, Dominance, Valence], ...] 형식의 감정 값 리스트
        filenames (list of str): 각 데이터에 해당하는 파일 이름 리스트
        n_clusters (int): 생성할 군집의 수 (기본값: 3)
    """
    # K-Means 클러스터링
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(emotion_data)

    # 각 군집에 따른 색상 설정
    colors = ['red', 'blue', 'green', 'purple', 'orange']
    cluster_colors = [colors[label] for label in cluster_labels]

    # 예측 값 분리
    arousal = [data[0] for data in emotion_data]
    dominance = [data[1] for data in emotion_data]
    valence = [data[2] for data in emotion_data]

    # 3D 그래프 생성
    fig = go.Figure()

    # 데이터 추가
    fig.add_trace(go.Scatter3d(
        x=arousal,
        y=dominance,
        z=valence,
        mode='markers+text',
        marker=dict(
            size=5,  # 점 크기
            color=cluster_colors,  # 군집 색상
            opacity=0.8
        ),
        text=filenames,  # 파일 이름 레이블
        textposition="top center"
    ))

    # 레이아웃 설정
    fig.update_layout(
        title=f'3D Emotion Graph with {n_clusters} Clusters',
        scene=dict(
            xaxis=dict(title='Arousal', range=[0, 1]),
            yaxis=dict(title='Dominance', range=[0, 1]),
            zaxis=dict(title='Valence', range=[0, 1]),
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    # 그래프 표시
    fig.show()

# 예제 데이터
emotion_predictions = [
    [0.5687486, 0.5290034, 0.5430766],  # temp2.wav
    [0.4568457, 0.6370042, 0.5901238],  # temp.wav
    [0.7238453, 0.4530021, 0.5201234],  # temp3.wav
    [0.1234577, 0.2370043, 0.1901239],  # temp4.wav
    [0.8234577, 0.8370043, 0.7901239],  # temp5.wav
]
file_names = ["temp2.wav", "temp.wav", "temp3.wav", "temp4.wav", "temp5.wav"]

# 군집화 및 시각화
cluster_and_plot_3d(emotion_predictions, file_names, n_clusters=3)